# Code propre

## I) Fonctions de bases

In [1]:
# Chargement des packages que nous allons utiliser dans ce script
import time 
import random
import re
import seaborn as sns
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from scipy.spatial import cKDTree
from scipy.stats import norm

# Fonction qui permet de récuperer le code html d'un page internet
# ATTENTION : IL FAUT MODIFIER L'USER AGENT !!!!!!

def get_page(urlpage):
    user_agent = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:122.0) Gecko/20100101 Firefox/122.0'}
    # Timer qui retard l'envoi de requête vers le site pour pas se faire ban
    time.sleep(0.2 + np.random.rand()/10)
    res = requests.get(urlpage, headers = user_agent)
    soup = BeautifulSoup(res.text, 'html.parser')
    return soup

def get_pagelink(urlpage):
    # Cette fonction sert à rechercher les différentes pages du site pour obtenir toutes les annonces du site lefigaro
    # L'idée ici est de regarder si la page à un bouton "suivant" pour aller à la prochaine page
    # Quand on a fait toutes les pages qui sont disponibles, link = "VIDE" donc on arrète car il n'y a pas de suivant
    # Et on stocke tous les liens dans "page" et dans ces pages on va récupérer les liens des annonces avec la fonction get_link()
    website = "https://immobilier.lefigaro.fr" 
    link = [] # On initialise un vecteur vide où on va stocker les liens des différences pages
    page = [urlpage] # La première page est celle qu'on a donner dans la fonction get_link()
    while link != "VIDE": # Tant que link n'est pas vide, c'est-à-dire tant qu'on est pas à la dernière page
        soup = get_page(urlpage) 
        text = soup.find_all("a", "router-link-active router-link-exact-active btn-pagination") # Récupération du html
        for i in range(len(text)): # Il y a 2 élements dans "text" : "Précédent" et "Suivant".
            if text[i].text == "Suivant": # On cherche l'element "Suivant"
                link = re.findall('href="(.*?)" ', str(text[i]))[0] # Et on cherche le lien
                urlpage = website + link # On merge le nom du site et la partie du lien spécifique à la page pour avoir le lien complet
                page.append(urlpage) # On ajoute le lien au vecteur "page" pour le stocker
                break
            else:
                link = "VIDE" # Si il n'y a de "Suivant", c'est que c'est la dernière page donc on arrète.
    return page

# Fonction qui nous permet de récupérer le lien des annonces des logements
# Les sites disponibles sont "orpi", "nexity" et "lefigaro"

def get_link(site):
    # On récupère les caractèristiques pour le site 'orpi'
    if (site=='orpi')==True:
        urlpage = 'https://www.orpi.com/location-immobiliere-strasbourg/louer-appartement/' # url
        a = "a" # la balise
        b = 'u-link-unstyled c-overlay__link' # la classe
        website = "https://www.orpi.com" # le lien du site web
        reg = 'href="(.*?)">\n<span' # la regular expression qui encadre notre lien
    # On récupère les caractéristiques pour le site 'nexity'
    if (site=='nexity')==True:
        urlpage = 'https://www.nexity.fr/annonces-immobilieres/location/appartement/tout/strasbourg+67' # url
        a = "div" # la balise
        b = 'product-card-content flex flex-column align-items-start' # la classe
        website = "https://www.nexity.fr" # le lien du site web
        reg = 'href="(.*?)" target' # la regular expression qui encadre notre lien
    
    # On récupère les caractèristiques pour le site 'lefigaro'
    if (site=='lefigaro')==True:
        urlpage = 'https://immobilier.lefigaro.fr/annonces/immobilier-location-appartement-strasbourg+67000.html' # url
        a = "a" # la balise
        b = 'content__link' # la classe
        website = "https://immobilier.lefigaro.fr" # le lien du site web
        reg = 'href="([^"]*)"' # la regular expression qui encadre notre lien
        
        # Comme il y a plusieurs pages sur le site "lefigaro", il faut prendre le lien des annonces pour toutes les pages
        page_link = get_pagelink(urlpage) # On récupère les liens des différentes pages de lefigaro
        links = [] # On initialise le vecteur où on va stocker tous les liens des annonces

        for i in page_link:
            soup = get_page(i)
            annonces = soup.find_all(a, class_= b) # On récupère le html de toutes les annonces sur cette page
            for i in range(len(annonces)):
                text = str(annonces[i]) # On met chaque annonce en chaine de caractère
                link = re.findall(reg, text)[0] # On cherche la regular expression qu'on a définit plus haut
                path = website + link # On regroupe le lien du site et le lien de l'annonce pour avoir le lien complet
                links.append(path) # On l'ajoute à notre vecteur "links"
    
    # Pour ces deux sites, pas besoin car il n'y a qu'une page
    if (site=='nexity' or site=='orpi')==True:
        # On charge la page d'acceuil ou toutes les annonces sont énumérées
        soup = get_page(urlpage) 
        # On recupère les infos dans le html
        annonces = soup.find_all(a, class_= b)
        # On initialise un vecteur vide pour stocker les liens des annonces
        links = []

        for i in range(len(annonces)):
            text = str(annonces[i]) #On met chaque annonce en chaine de caractère
            link = re.findall(reg, text)[0] # On cherche la regular expression qu'on a définit plus haut
            path = website + link # On regroupe le nom du site et le nom de l'annonce
            links.append(path) # On l'ajoute à notre vecteur "links"
        
    return links

## II) ORPI

In [2]:
# a) Type : appartement ou maison
def type_orpi(soup):
    text = soup.find_all("span", class_='u-block@sm u-block@md-plus')[0].text
    if text.find("Appartement") != -1:
        typ = "Appartement"
    elif text.find("Maison") != -1:
        typ = "Maison"
    return [typ]

# b) Ville : Strasbourg et alentour
def ville_orpi(soup):
    text = soup.find("span", class_='u-h3 u-ml-xs u-text-normal').text
    return [text]

# c) Loyer
def loyer_orpi(soup): 
    text = soup.find("ul", class_='u-list-unstyled u-text-xs u-mt-xs u-color-text-grey').find_all('li')
    for i in range(len(text)):
        element = text[i].text
        if "Loyer" in element:
            loyer = [float(''.join(re.findall(r'\d', element)))]
    return loyer

# d) Charges
def charges_orpi(soup): 
    text = soup.find("ul", class_='u-list-unstyled u-text-xs u-mt-xs u-color-text-grey').find_all('li')
    for i in range(len(text)):
        element = text[i].text
        if "Provisions" in element:
            charges = [float(''.join(re.findall(r'\d', element)))]
    return charges

# e) Honoraires
def honoraires_orpi(soup): 
    text = soup.find("ul", class_='u-list-unstyled u-text-xs u-mt-xs u-color-text-grey').find_all('li')
    honoraires = [float('nan')]
    for i in range(len(text)):
        element = text[i].text
        if "Honoraire" in element:
            honoraires = [float(''.join(re.findall(r'\d', element)))]
    return honoraires

# f) Pièces
def piece_orpi(soup):
    text = soup.find_all(class_='u-flex u-flex-cross-center')
    piece = [float('nan')]
    for i in range(len(text)):
        if "pièce" in text[i].find("span").text:
            piece = [float(re.findall(r'\d+', text[i].find("span").text)[0])]
    if (piece == 'nan')==True:
        text = soup.find_all(class_='u-block@sm u-block@md-plus')[1].text.split()
        for i in range(len(text)):
            if "pièce" in text[i]:
                piece = [float(text[i-1])]
    return piece

# g) Surface
def surface_orpi(soup):
    text = soup.find_all(class_='u-flex u-flex-cross-center')
    surface = [float('nan')]
    for i in range(len(text)):
        if "Surface" in text[i].find("span").text:
            surface = [float(re.findall(r'\d+', text[i].find("span").text)[0])]
    if (surface == 'nan')==True:
        text = soup.find_all(class_='u-block@sm u-block@md-plus')[1].text.split()
        for i in range(len(text)):
            if "m2" in text[i]:
                surface = [float(text[i-1])]
    return surface

# h) Meublé
def meuble_orpi(soup):
    text = soup.find_all(class_='u-flex u-flex-cross-center')
    meuble = [float(0)]
    for i in range(len(text)-1):
        element = text[i].find("span").text
        if "Meublé" in element:
            meuble = [float(1)]
    return meuble

# i) Ascenseur
def ascenseur_orpi(soup):
    text = soup.find_all(class_='u-flex u-flex-cross-center')
    ascenseur = [float(0)]
    for i in range(len(text)-1):
        element = text[i].find("span").text
        if "Ascenseur" in element:
            ascenseur = [float(1)]
    return ascenseur

# j) Balcon
def balcon_orpi(soup):
    text = soup.find_all(class_='u-flex u-flex-cross-center')
    balcon = [float(0)]
    for i in range(len(text)-1):
        element = text[i].find("span").text
        if "Balcon" in element:
            balcon = [float(1)]
    return balcon

# k) Terrasse
def terrasse_orpi(soup):
    text = soup.find_all(class_='u-flex u-flex-cross-center')
    terrasse = [float(0)]
    for i in range(len(text)-1):
        element = text[i].find("span").text
        if "Terrasse" in element:
            terrasse = [float(1)]
    return terrasse

# l) Cave
def cave_orpi(soup):
    text = soup.find_all(class_='u-flex u-flex-cross-center')
    cave = [float(0)]
    for i in range(len(text)-1):
        element = text[i].find("span").text
        if "Cave" in element:
            cave = [float(1)]
    return cave

# m) Parking
def parking_orpi(soup):
    text = soup.find_all(class_='u-flex u-flex-cross-center')
    parking = [float(0)]
    for i in range(len(text)-1):
        element = text[i].find("span").text
        if "Parking" in element or "Garage" in element:
            parking = [float(1)]
    return parking

# n) Terrain
def terrain_orpi(soup):
    text = soup.find_all(class_='u-flex u-flex-cross-center')
    terrain = [float(0)]
    for i in range(len(text)-1):
        element = text[i].find("span").text
        if "Jardin" in element or "Terrain" in element:
            terrain = [float(1)]
    return terrain


# o) Quartier
def quartier_orpi(soup):
    quartiers = ["meinau", "neustadt", 'poteries', "esplanade", "petite france", "cronenbourg", "koenigshoffen", "halles", "neudorf", "robertsau", "gare", "musau", "orangerie", "krutenau", "forêt noire", 'centre-ville', "port du rhin", "hautepierre", "contades"]
    text = soup.find_all("h2", class_='u-h3')
    if ville_orpi(soup) == ["Strasbourg"]:
        for i in range(len(text)):
            element = text[i].text
            if "Quartier" in element:
                localisation = re.findall(r'Quartier (.*?) à', element)
                for j in range(len(quartiers)):
                    if localisation[0].lower().find(quartiers[j]) != -1:
                        quartier = [quartiers[j].capitalize()]
    else:
        quartier = [ville_orpi(soup)]
    return quartier

# Fonction qui nous donne toutes les caractéristiques de l'annonce orpi à l'aide des fonction précédente 
def get_orpi(urlpage):
    user_agent = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:122.0) Gecko/20100101 Firefox/122.0'}
    soup = get_page(urlpage)
    link = [urlpage]
    appart = type_orpi(soup) + ville_orpi(soup) + quartier_orpi(soup) + loyer_orpi(soup) + charges_orpi(soup) + honoraires_orpi(soup) + piece_orpi(soup) + surface_orpi(soup) + meuble_orpi(soup) + ascenseur_orpi(soup) + balcon_orpi(soup) + terrasse_orpi(soup) + cave_orpi(soup) + parking_orpi(soup) + terrain_orpi(soup) + link
    return appart


## III) NEXITY

In [3]:
# a) type
def type_nexity(soup):    
    text = soup.find_all("div", class_='d-flex align-items-center')
    for i in range(len(text)):
        element = text[i].text.lower().split()
        if "type" in element:
            typ = [element[len(element)-1].capitalize()]
            break
    return typ

# b) ville
def ville_nexity(soup):
    ville = [soup.find("span", class_='city').text]
    return ville

# c) Quartier
def quartier_nexity(soup):
    if ville_nexity(soup) == ["Strasbourg"]:
        quartiers = ["meinau", "neustadt", 'poteries', "esplanade", "petite france", "cronenbourg", "koenigshoffen", "halles", "neudorf", "robertsau", "gare", "musau", "orangerie", "krutenau", "forêt noire", 'centre-ville', "port du rhin", "hautepierre", "contades"]
        text = soup.find("div", class_='description text_body_1 mt-2').text.lower().split()
        quartier = []
        for i in range(len(quartiers)):
            for j in range(len(text)):
                if quartiers[i] == text[j]:
                    quartier = [quartiers[i].capitalize()]
                    break
        if quartier == []:
            quartier = ["nan"]
    else:
        quartier = ville_nexity(soup)
    return quartier

# d) Loyer
def loyer_nexity(soup):    
    text = soup.find_all("div", class_='d-flex align-items-center')
    for i in range(len(text)):
        element = text[i].text.lower().split()
        if "loyer" in element:
            loyer = [float(element[len(element)-1])]
            break
    return loyer

# e) Charges
def charges_nexity(soup):    
    text = soup.find_all("div", class_='d-flex align-items-center')
    for i in range(len(text)):
        element = text[i].text.lower().split()
        if "charges" in element:
            charges = [float(element[len(element)-1])]
            break
    return charges

# f) Honoraires
def honoraires_nexity(soup):
    text = soup.find("div", class_="block-characteristiques--bareme")
    text = text.text
    text = re.sub(r'\s+', ' ', text)
    hono1 = float(re.findall(r"Honoraires d'organisation de la visite.*?(\d+\.\d+)", text)[0])
    hono2 = float(re.findall(r"Honoraires de réalisation d'état des lieux.*?(\d+)", text)[0])
    honoraires = [hono1 + hono2]
    return honoraires

# g) Pièces
def piece_nexity(soup):    
    piece = [float("nan")]
    text = soup.find_all("div", class_='d-flex align-items-center')
    for i in range(len(text)):
        element = text[i].text.lower().split()
        if "pièce(s)" in element:
            piece = [float(element[len(element)-1])]
            break
    return piece

# h) Surface
def surface_nexity(soup):    
    text = soup.find_all("div", class_='d-flex align-items-center')
    for i in range(len(text)):
        element = text[i].text.lower().split()
        if "surface" in element:
            surface = [float(re.findall(r'\d+', element[len(element)-1])[0])]
            break
    return surface
                               
# i) Meublé
def meuble_nexity(soup):
    text = soup.find_all("div", class_='flap flap--not-new')
    if (text == [])==True:
        meuble = [float(0)]
    else:
        text = text[0].text
        if str(text)=='location meublée':
            meuble = [float(1)]
        else:
            meuble = [float(0)]
    return meuble            
                           
# j) Ascenseur
def ascenseur_nexity(soup):    
    text = soup.find_all("div", class_='d-flex align-items-center')
    for i in range(len(text)):
        element = text[i].text.lower().split()
        if "ascenseur" in element:
            if element[len(element)-1] == "oui":
                ascenseur = [float(1)]
            else:
                ascenseur = [float(0)]
            break
    return ascenseur

# k) Balcon
def balcon_nexity(soup, carac):    
    if carac not in ["balcon"]:
        raise Exception("Mauvaise caractéristique")
    text = soup.find_all("div", class_='d-flex align-items-center')
    for i in range(len(text)):
        element = text[i].text.lower().split()
        if "balcon" in element:
            if element[len(element)-1] == "oui":
                balcon = [float(1)]
            else:
                balcon = [float(0)]
            break
    return balcon

# l) Terrasse
def terrasse_nexity(soup):    
    text = soup.find_all("div", class_='d-flex align-items-center')
    for i in range(len(text)):
        element = text[i].text.lower().split()
        if "terrasse" in element:
            if element[len(element)-1] == "oui":
                terrasse = [float(1)]
            else:
                terrasse = [float(0)]
            break
    return terrasse

# m) Cave
def cave_nexity(soup):    
    text = soup.find_all("div", class_='d-flex align-items-center')
    for i in range(len(text)):
        element = text[i].text.lower().split()
        if "cave" in element:
            if element[len(element)-1] == "oui":
                cave = [float(1)]
            else:
                cave = [float(0)]
            break
    return cave

# n) Parking
def parking_nexity(soup):    
    text = soup.find_all("div", class_='d-flex align-items-center')
    for i in range(len(text)):
        element = text[i].text.lower().split()
        if "stationnement" in element:
            if element[len(element)-1] == "non":
                parking = [float(0)]
            else:
                parking = [float(1)]
            break
    return parking

# o) Terrain
def terrain_nexity(soup):    
    text = soup.find_all("div", class_='d-flex align-items-center')
    for i in range(len(text)):
        element = text[i].text.lower().split()
        if "terrain" in element:
            if element[len(element)-1] == "non":
                terrain = [float(0)]
            else:
                terrain = [float(1)]
            break
    return terrain

# Fonction qui nous donne toutes les caractéristiques d'une annonce nexity à l'aide des fonction précédente 
def get_nexity(urlpage):
    user_agent = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:122.0) Gecko/20100101 Firefox/122.0'}
    soup = get_page(urlpage)
    link = [urlpage]
    appart = type_nexity(soup) + ville_nexity(soup) + quartier_nexity(soup) +  loyer_nexity(soup)  + charges_nexity(soup) + honoraires_nexity(soup) + piece_nexity(soup) + surface_nexity(soup) + meuble_nexity(soup) + ascenseur_nexity(soup) + balcon_nexity(soup) + terrasse_nexity(soup) + cave_nexity(soup) + parking_nexity(soup) + terrain_nexity(soup) + link
    return appart

## IV) LEFIGARO

In [53]:
# a) Type
def type_lefigaro(soup):
    text = soup.find_all('h1')[0].text.lower()
    if 'appartement' in text or 'studio' in text or 'location' in text:
        typ=['Appartement']
    elif 'maison' in text:
        typ=['Maison']
    return typ

# b) Ville
def ville_lefigaro(soup):
    text = soup.find_all('h1')[0].text
    ville = [re.search(r'à (\w+)', text).group(1)]
    return ville

# c) Quartier
def quartier_lefigaro(soup):
    quartiers = ["meinau", "neustadt", 'poteries', "esplanade", "petite france", "cronenbourg", "koenigshoffen", "halles", "neudorf", "robertsau", "gare", "musau", "orangerie", "krutenau", "forêt noire", 'centre-ville', "port du rhin", "hautepierre", "contades"]
    text = soup.find_all("p", class_='truncated-description')[0].text.lower()
    quartier=0
    if ville_lefigaro(soup)==["Strasbourg"]:
        for i in range(len(quartiers)):
            if text.lower().find(quartiers[i]) != -1:
                quartier = [quartiers[i].capitalize()]
    else:    
        for i in range(len(quartiers)):
                quartier = [ville_lefigaro(soup)]
    if quartier == 0:
        quartier = ["nan"]
    return quartier

# d) Loyer et charges
def loyer_charges_lefigaro(soup):
    loyer = soup.find_all('span', class_='price')[0].text
    loyer = float(re.sub(r'[^\d.]', '', loyer))
    charges = soup.find_all('span', class_='about-price-fees-label')
    if (charges == [])==True:
        charges = 0
    else:
        charges = charges[0].text
        charges = float(re.search(r'\d+', charges).group())
    if "CC" in (soup.find_all('span', class_='price')[0].text):
        loyer = loyer - charges
    return [loyer, charges]

# e) Honoraires
def honoraires_lefigaro(soup):
    text = soup.find_all('li', class_='item-about-price')
    honoraires = [float('nan')]
    for i in range(len(text)):
        if "honoraires" in text[i].text.lower():
            if "non communiqué" in text[i].text.lower():
                honoraires = [float('nan')]
            else:
                honoraires = [float(re.search(r'\b\d+\b',text[2].text.lower())[0])]
    return honoraires

# f) Pièces
def piece_lefigaro(soup):
    text= soup.find_all('span', class_='feature')
    for i in range(len(text)):
        element = text[i].text.split()
        if "pièces" in element or 'pièce' in element:
            piece = [float(re.match(r'([\d.]+)', element[0]).group())]
    return piece


# g) Surface
def surface_lefigaro(soup):
    text = soup.find_all('span', class_='feature')
    for i in range(len(text)):
        element = text[i].text.split()
        if "surface" in element:
            surface = [float(re.match(r'([\d.]+)', element[0]).group())]
    return surface

# h) Meublé
def meuble_lefigaro(soup):
    text = soup.find_all("p", class_='truncated-description')[0].text.lower()
    if "meublé" in text:
        meuble = [float(1)]
    else:
        meuble = [float(0)]
    return meuble

# i) Ascenseur
def ascenseur_lefigaro(soup):
    text = soup.find_all("li", class_='options')
    element = [float(0)]
    if text != []:
        for i in range(len(text)):
            if ("ascenseur" == text[i].text.lower())==True:
                element = [float(1)]
    return element

# j) Balcon
def balcon_lefigaro(soup):
    text = soup.find_all("li", class_='options')
    element = [float(0)]
    if text != []:
        for i in range(len(text)):
            if ("balcon" == text[i].text.lower())==True:
                element = [float(1)]
    return element

# k) Terrasse
def terrasse_lefigaro(soup):
    text = soup.find_all("li", class_='options')
    element = [float(0)]
    if text != []:
        for i in range(len(text)):
            if ("terrasse" == text[i].text.lower())==True:
                element = [float(1)]
    return element

# l) Cave
def cave_lefigaro(soup):
    text = soup.find_all("li", class_='options')
    element = [float(0)]
    if text != []:
        for i in range(len(text)):
            if ("cave" == text[i].text.lower())==True:
                element = [float(1)]
    return element

# m) Parking
def parking_lefigaro(soup):
    text = soup.find_all("li", class_='options')
    element = [float(0)]
    if text != []:
        for i in range(len(text)):
            if ("parking" == text[i].text.lower())==True:
                element = [float(1)]
    return element

# n) Terrain
def terrain_lefigaro(soup):
    text = soup.find_all("li", class_='options')
    element = [float(0)]
    if text != []:
        for i in range(len(text)):
            if ("jardin" == text[i].text.lower())==True:
                element = [float(1)]
    return element

def get_lefigaro(urlpage):
    user_agent = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:122.0) Gecko/20100101 Firefox/122.0'}
    soup = get_page(urlpage)
    link = [urlpage]
    appart = type_lefigaro(soup) + ville_lefigaro(soup) + quartier_lefigaro(soup) + loyer_charges_lefigaro(soup) + honoraires_lefigaro(soup) + piece_lefigaro(soup) + surface_lefigaro(soup) + meuble_lefigaro(soup) + ascenseur_lefigaro(soup) + balcon_lefigaro(soup) + terrasse_lefigaro(soup) + cave_lefigaro(soup) + parking_lefigaro(soup) + terrain_lefigaro(soup) + link
    return appart

## V) DATA

In [51]:
# Fonction qui reprend toutes les fonctions précédentes pour faire une fonction qui fait tout !
def get_appart(site):
    links = get_link(site)
    var = ["Type", "Ville", "Quartier", "Loyer", "Charges", "Honoraires", "Pièces", "Surface", "Meublé", "Ascenseur", "Balcon", "Terrasse", "Cave", "Parking", "Terrain", "Liens"]
    data = []
    if (site=="orpi")==True:
        for i in range(len(links)):
            info = get_orpi(links[i])
            data.append(info)
    if (site=="nexity")==True:
        for i in range(len(links)):
            if (links[i].find("residence_etudiant") != -1)==False:
                info = get_nexity(links[i])
                data.append(info)
    if (site=="lefigaro")==True:
        for i in range(len(links)):
            info = get_lefigaro(links[i])
            data.append(info)
    data = pd.DataFrame(data, columns=var)
    return data  

def import_data(site):
    t = time.time()
    if site != "all":
        dt = get_appart(site)
    else:
        dt = pd.concat([get_appart("orpi"), get_appart("nexity"), get_appart("lefigaro")], ignore_index=True)
    dt = dt.dropna(subset=["Loyer", "Surface", 'Charges', 'Pièces', 'Terrain', 'Meublé', 'Ascenseur', 'Balcon', 'Terrasse','Cave','Parking'])
    dt = dt.reset_index(drop=True)
    n=len(dt)-1
    val_double_i = []
    val_double_j = []
    for i in range(0,n):
        for j in range(i+1,n+1):
            if (dt.iloc[i, [1,2,4,5,7,8]].tolist() == dt.iloc[j, [1,2,4,5,7,8]].tolist())==True:
                val_double_i.append(i)
                val_double_j.append(j)
    val_double = []
    for i in range(len(val_double_i)):
        if dt.iloc[val_double_i[i],6] > dt.iloc[val_double_j[i],6]:
            val_double.append(val_double_i[i])
        elif dt.iloc[val_double_i[i],6] < dt.iloc[val_double_j[i],6] or dt.iloc[val_double_i[i],6] == dt.iloc[val_double_j[i],6]:
            val_double.append(val_double_j[i])
        elif dt.iloc[val_double_i[i],6] == 'nan' and dt.iloc[val_double_j[i],6] != 'nan':
            val_double.append(val_double_i[i])
        elif dt.iloc[val_double_i[i],6] != 'nan' and dt.iloc[val_double_j[i],6] == 'nan':
            val_double.append(val_double_j[i])
        else:
            val_double.append(val_double_j[i])   
    dt = dt.drop(list(np.unique(val_double)))
    dt = dt.reset_index(drop=True)
    print((time.time() - t)/60)
    return dt

In [55]:
dt = import_data("all")
dt

3.6066736459732054


,Type,Ville,Quartier,Loyer,Charges,Honoraires,Pièces,Surface,Meublé,Ascenseur,Balcon,Terrasse,Cave,Parking,Terrain,Liens
0,Appartement,Strasbourg,Neudorf,459.0,50.0,NaN,1.0,19.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,https://www.orpi.com/annonce-location-appartem...
1,Appartement,Strasbourg,Robertsau,1000.0,50.0,1117.0,4.0,123.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,https://www.orpi.com/annonce-location-appartem...
2,Appartement,Strasbourg,Forêt noire,585.0,180.0,NaN,2.0,48.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,https://www.orpi.com/annonce-location-appartem...
3,Appartement,Strasbourg,Robertsau,540.0,45.0,585.0,2.0,45.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,https://www.orpi.com/annonce-location-appartem...
4,Appartement,Strasbourg,Neudorf,337.0,110.0,364.0,1.0,28.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,https://www.orpi.com/annonce-location-appartem...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162,Appartement,Strasbourg,nan,880.0,0.0,NaN,2.0,37.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,https://immobilier.lefigaro.fr/annonces/annonc...
163,Appartement,Strasbourg,nan,610.0,0.0,NaN,1.0,20.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,https://immobilier.lefigaro.fr/annonces/annonc...
164,Appartement,Strasbourg,nan,850.0,0.0,NaN,2.0,45.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,https://immobilier.lefigaro.fr/annonces/annonc...
165,Appartement,Strasbourg,nan,674.0,0.0,NaN,3.0,66.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,https://immobilier.lefigaro.fr/annonces/annonc...


## VI) KNN

In [58]:
def get_estimation(Information):
    a=dt[["Surface", 'Charges', 'Pièces', 'Terrain', 'Meublé', 'Ascenseur', 'Balcon', 'Terrasse','Cave', 'Parking']]
    b=dt["Loyer"]
    key = list(Information.keys())
    X = []
    for i in range(len(key)):
        if (Information.get(key[i])=="Oui")==True:
            ad = 1
        elif (Information.get(key[i])=="Non")==True:
            ad = 0
        else :
            ad = Information.get(key[i])
        X.append(ad)
    model = KNeighborsClassifier(n_neighbors=4)
    model.fit(a,b)
    distances, indices = model.kneighbors([X[1:]])
    loy = []
    for i in range(len(indices[0])):    
        obs = dt.iloc[indices[0][i]]["Loyer"]
        loy.append(obs)
    loy = np.array(loy)
    z = distances.sum()- distances[0]
    sum_z = z.sum()
    pond = z/sum_z
    estimation = (loy*pond).sum()
    # Intervalle de confiance
    lower_bound = (loy*pond).sum() -  norm.ppf((1.95) / 2) * (np.std(loy) / np.sqrt(len(loy)))
    upper_bound = (loy*pond).sum() + norm.ppf((1.95) / 2) * (np.std(loy) / np.sqrt(len(loy)))
    IC = [int(lower_bound.round()), int(upper_bound.round())]
    print("Estimation du prix : ", estimation.round(), "€")
    print("Interval de confiance du prix : ", IC)
    
def get_annonce(Information):
    a=dt[["Loyer", "Surface", 'Charges', 'Pièces', 'Terrain', 'Meublé', 'Ascenseur', 'Balcon', 'Terrasse','Cave','Parking']]
    b=dt["Liens"]
    key = list(Information.keys())
    X = []
    for i in range(len(key)):
        if (Information.get(key[i])=="Oui")==True:
            ad = 1
        elif (Information.get(key[i])=="Non")==True:
            ad = 0
        else :
            ad = Information.get(key[i])
        X.append(ad)
    model = KNeighborsClassifier(n_neighbors=4)
    model.fit(a,b)
    distances, indices = model.kneighbors([X])
    la = dt.iloc[indices[0]][["Liens", "Loyer", "Surface", "Pièces", "Quartier"]]
    


    for i in range(len(la)):
        if la.iloc[i][4]=="nan":
            quart = "Non renseigné"
        else:
            quart = la.iloc[i][4]
        print("Annonce",i+1,":")
        print("Quartier : ", quart)
        print("Loyer : ", int(la.iloc[i][1]), "€\n" "Surface : ", la.iloc[i][2], "m2","\n" "Pièces : ", int(la.iloc[i][3]),"\n" "Lien : ", la.iloc[i][0])
        print("\n")

In [2]:
Information = {"Loyer" : 1000,      # En €
              "Surface" : 80,      # En m²
              "Charges": 100,       # En €
              "Pièces" : 4,        # En nombre
              "Terrain" : 0,       # En m²
              "Meublé" : "Non",    # "Oui" ou "Non"
              "Ascenseur" : "Non", # "Oui" ou "Non"
              "Balcon" : "Oui",    # "Oui" ou "Non"
              "Terrasse" : "Non",  # "Oui" ou "Non"
              "Cave" : "Non",      # "Oui" ou "Non"
              "Parking" : "Oui",   # "Oui" ou "Non"
              }

In [59]:
get_estimation(Information)

Estimation du prix :  972.0 €
Interval de confiance du prix :  [848, 1097]


C:\Users\busch\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


In [60]:
impo
get_annonce(Information)

Annonce 1 :
Quartier :  Robertsau
Loyer :  1000 €
Surface :  123.0 m2 
Pièces :  4 
Lien :  https://www.orpi.com/annonce-location-appartement-t4-strasbourg-67000-26a87abe-0fc2-4877-b567-264bf0d02b39/


Annonce 2 :
Quartier :  Gare
Loyer :  997 €
Surface :  123.0 m2 
Pièces :  4 
Lien :  https://www.orpi.com/annonce-location-appartement-t4-strasbourg-67000-7ce1b01e-db95-46b4-95e8-2b5c5fcd1851/


Annonce 3 :
Quartier :  Gare
Loyer :  997 €
Surface :  123.29 m2 
Pièces :  4 
Lien :  https://immobilier.lefigaro.fr/annonces/annonce-65686638.html


Annonce 4 :
Quartier :  Neudorf
Loyer :  940 €
Surface :  100.0 m2 
Pièces :  4 
Lien :  https://www.orpi.com/annonce-location-appartement-t4-strasbourg-67100-2819ccf2-579d-4726-8bd2-c45a7ff61223/




C:\Users\busch\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


In [63]:
print("Entrez votre prénom : ", end="")
prenom = input()
print("Je m'appelle ", prenom)

Entrez votre prénom : 

 Adrien


Je m'appelle  Adrien


In [62]:
input("Entrez votre prénom : ")

Entrez votre prénom :  Adrien


'Adrien'

In [64]:
prenom = "Albert"
chaine = prenom + " ! Combien voulez-vous de cartes ?"
nb_cartes = int(input(chaine))  

Albert ! Combien voulez-vous de cartes ? 5


In [1]:
import estimation_fonctions

In [4]:
estimation_fonctions.get_estimation(Information)

NameError: name 'dt' is not defined